<a href="https://colab.research.google.com/gist/KKhouma/fbd0b12f1456ea40224cde4180c24626/corn-disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

Import zip

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# we use google colab and store the data in shared drive. different source will have different method
# dont forget to mount drive first
local_zip = '/content/drive/Shareddrives/Capstone C23-PC599/Machine Learning/corndisease.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [6]:
source_path = '/tmp/jagung'

source_path_bercak = os.path.join(source_path, 'bercak')
source_path_hawar = os.path.join(source_path, 'hawar')
source_path_karat = os.path.join(source_path, 'karat')
source_path_sehat = os.path.join(source_path, 'sehat')


print(f"ada {len(os.listdir(source_path_bercak))} gambar bercak daun.")
print(f"ada {len(os.listdir(source_path_hawar))} gambar hawar daun.")
print(f"ada {len(os.listdir(source_path_karat))} gambar karat daun.")
print(f"ada {len(os.listdir(source_path_sehat))} gambar daun sehat.")


ada 513 gambar bercak daun.
ada 985 gambar hawar daun.
ada 1192 gambar karat daun.
ada 1162 gambar daun sehat.


Make directory for split training and validation

In [7]:
root_dir = '/tmp/corn_disease'

if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

def create_dir(root_path):
  os.mkdir('/tmp/corn_disease')
  os.mkdir('/tmp/corn_disease/training')
  os.mkdir('/tmp/corn_disease/validation')
  os.mkdir('/tmp/corn_disease/training/bercak')
  os.mkdir('/tmp/corn_disease/training/hawar')
  os.mkdir('/tmp/corn_disease/training/karat')
  os.mkdir('/tmp/corn_disease/training/sehat')
  os.mkdir('/tmp/corn_disease/validation/bercak')
  os.mkdir('/tmp/corn_disease/validation/hawar')
  os.mkdir('/tmp/corn_disease/validation/karat')
  os.mkdir('/tmp/corn_disease/validation/sehat') 

try:
  create_dir(root_path=root_dir)
except FileExistsError:
  print("direktori sudah ada")

In [8]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/tmp/corn_disease/validation
/tmp/corn_disease/training
/tmp/corn_disease/validation/sehat
/tmp/corn_disease/validation/bercak
/tmp/corn_disease/validation/hawar
/tmp/corn_disease/validation/karat
/tmp/corn_disease/training/sehat
/tmp/corn_disease/training/bercak
/tmp/corn_disease/training/hawar
/tmp/corn_disease/training/karat


Split Data

In [9]:
def split_data(source_dir, training_dir, validation_dir, split_size):
  files = []
  for filename in os.listdir(source_dir):
    file = source_dir + filename
    if os.path.getsize(file) > 0:
      files.append(filename)
    else:
      print(filename + 'is zero length, so ignoring.')
      
  training_length = int(len(files) * split_size)
  validation_length = int(len(files) - training_length)
  shuffled_set = random.sample(files, len(files))
  training_set = shuffled_set[0:training_length]
  validation_set = shuffled_set[:validation_length]

  for filename in training_set:
    this_file = source_dir + filename
    destination = training_dir + filename
    copyfile(this_file, destination)    

  for filename in validation_set:
    this_file = source_dir + filename
    destination = validation_dir + filename
    copyfile(this_file, destination)

In [10]:
#source file
source_bercak_dir = "/tmp/jagung/bercak/"
source_hawar_dir = "/tmp/jagung/hawar/"
source_karat_dir = "/tmp/jagung/karat/"
source_sehat_dir = "/tmp/jagung/sehat/"

#define train and validation dir
training_dir = "/tmp/corn_disease/training"
validation_dir = "/tmp/corn_disease/validation"

#define file bercak jagung
training_bercak_dir = os.path.join(training_dir, "bercak/")
validation_bercak_dir = os.path.join(validation_dir, "bercak/")

#define file hawar jagung
training_hawar_dir = os.path.join(training_dir, "hawar/")
validation_hawar_dir = os.path.join(validation_dir, "hawar/")

#define file karat jagung
training_karat_dir = os.path.join(training_dir, "karat/")
validation_karat_dir = os.path.join(validation_dir, "karat/")

#define file jagung sehat
training_sehat_dir = os.path.join(training_dir, "sehat/")
validation_sehat_dir = os.path.join(validation_dir, "sehat/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(training_bercak_dir)) > 0:
  for file in os.scandir(training_bercak_dir):
    os.remove(file.path)
if len(os.listdir(validation_bercak_dir)) > 0:
  for file in os.scandir(validation_bercak_dir):
    os.remove(file.path)
if len(os.listdir(training_hawar_dir)) > 0:
  for file in os.scandir(training_hawar_dir):
    os.remove(file.path)
if len(os.listdir(validation_hawar_dir)) > 0:
  for file in os.scandir(validation_hawar_dir):
    os.remove(file.path)
if len(os.listdir(training_karat_dir)) > 0:
  for file in os.scandir(training_karat_dir):
    os.remove(file.path)
if len(os.listdir(validation_karat_dir)) > 0:
  for file in os.scandir(validation_karat_dir):
    os.remove(file.path)
if len(os.listdir(training_sehat_dir)) > 0:
  for file in os.scandir(training_sehat_dir):
    os.remove(file.path)
if len(os.listdir(validation_sehat_dir)) > 0:
  for file in os.scandir(validation_sehat_dir):
    os.remove(file.path)
# Define proportion of images used for training
split_size = .9

split_data(source_bercak_dir, training_bercak_dir, validation_bercak_dir, split_size)
split_data(source_hawar_dir, training_hawar_dir, validation_hawar_dir, split_size)
split_data(source_karat_dir, training_karat_dir, validation_karat_dir, split_size)
split_data(source_sehat_dir, training_sehat_dir, validation_sehat_dir, split_size)

# Your function should perform copies rather than moving images so original directories should contain unchanged images
print(f"\n\nOriginal bercak directory has {len(os.listdir(source_bercak_dir))} images")
print(f"Original hawar directory has {len(os.listdir(source_hawar_dir))} images")
print(f"Original karat directory has {len(os.listdir(source_karat_dir))} images")
print(f"Original sehat directory has {len(os.listdir(source_sehat_dir))} images\n")

print(f"There are {len(os.listdir(training_bercak_dir))} images of bercak for training")
print(f"There are {len(os.listdir(training_hawar_dir))} images of hawar for training")
print(f"There are {len(os.listdir(training_karat_dir))} images of karat for training")
print(f"There are {len(os.listdir(training_sehat_dir))} images of sehat for training")
print(f"There are {len(os.listdir(validation_bercak_dir))} images of bercak for validation")
print(f"There are {len(os.listdir(validation_hawar_dir))} images of hawar for validation")
print(f"There are {len(os.listdir(validation_karat_dir))} images of karat for validation")
print(f"There are {len(os.listdir(validation_sehat_dir))} images of sehat for validation")





Original bercak directory has 513 images
Original hawar directory has 985 images
Original karat directory has 1192 images
Original sehat directory has 1162 images

There are 461 images of bercak for training
There are 886 images of hawar for training
There are 1072 images of karat for training
There are 1045 images of sehat for training
There are 52 images of bercak for validation
There are 99 images of hawar for validation
There are 120 images of karat for validation
There are 117 images of sehat for validation


In [19]:
!git config --global user.email "rohmankho@gmail.com"
!git config --global user.name "KKhouma"

In [20]:
!echo "# Machine-Learning-Model" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin git@github.com:KKhouma/CNN-corn-test-.git
!git push -u origin main

Reinitialized existing Git repository in /content/.git/
[main f23d5a8] first commit
 1 file changed, 1 insertion(+)
fatal: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address
